## 决策树模型实现冬小麦提取
依据作物在不同物候期内卫星影像的光谱存在差异的特征，可建立冬小麦提取算法，进行像元尺度冬小麦提取。

### 初始化环境

In [1]:
import aie

aie.Authenticate()
aie.Initialize()

### 指定需要检索的区域

In [2]:
region = aie.FeatureCollection('China_City') \
            .filter(aie.Filter.eq('city', '亳州市'))

### 影像检索

In [3]:
# 指定检索数据集，可设置检索的空间和时间范围，以及属性过滤条件（如云量过滤等）
def getl8_ndvi(startdate,enddate):
    dataset = aie.ImageCollection('LANDSAT_LC08_C02_T1_L2') \
                 .filterBounds(region) \
                 .filterDate(startdate, enddate) 
                 # .filter(aie.Filter.lte('eo:cloud_cover', 20.0))
    ndvi =  dataset.map(get_ndvi)  
    return ndvi

def get_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4'])
    return ndvi  

# 黄淮海地区冬小麦典型物候期。播种期 10-11月，旺长期3-4月，成熟期5-6月
NDVI_median = getl8_ndvi('2017-10-11', '2017-11-10').median().clipToCollection(region) 
NDVI_max = getl8_ndvi('2018-03-20', '2018-04-20').max().clipToCollection(region) 
NDVI_min = getl8_ndvi('2018-05-20', '2018-06-30').min().clipToCollection(region) 

# 冬小麦提取规则集
mask1 = NDVI_max.gt(aie.Image.constant(0.33))     # 过滤水体、不透水面、裸地等非植被区域
mask2 = NDVI_median.lt(aie.Image.constant(0.50))  # 过滤森林、春播作物

mask3 = NDVI_max.lte(aie.Image.constant(0.48))    # < 0.48存在
mask4 = NDVI_max.gt(aie.Image.constant(0.48))     # 区分植被区域和非植被区域，> 0.48 为植被区域


mask5 = NDVI_min.gt(aie.Image.constant(-0.12))    # 筛选冬小麦区域，成熟/收获期冬小麦植被指数下降
mask6 = NDVI_min.lt(aie.Image.constant(0.17))   
mask7 = NDVI_min.lt(NDVI_max.add(aie.Image.constant(0.5)))

wheat1 = mask1.And(mask2).And(mask3).And(mask5).And(mask6)
wheat2 = mask1.And(mask2).And(mask4).And(mask5).And(mask7)
wheat  = wheat1.add(wheat2).where((wheat1.add(wheat2)).gt(aie.Image.constant(0)),aie.Image.constant(1))

### 数据可视化

In [4]:
map = aie.Map(
    center=region.getCenter(),
    height=800,
    zoom=7
)

vis_params = {
    'color': '#00FF00'
}

map.addLayer(
    region,
    vis_params,
    'region',
    bounds=region.getBounds()
)

mask_vis  = {
    'min': 0,
    'max': 1,
    'palette': ['#ffffff', '#008000']    # 0:白色, 1:绿色
}

ndvi_vis  = {
    'min': -0.2,
    'max': 0.6,
    'palette': ['#d7191c', '#fdae61', '#ffffc0', '#a6d96a', '#1a9641']
}

map.addLayer(NDVI_median,ndvi_vis, 'NDVI_median', bounds=region.getBounds())
map.addLayer(NDVI_max,ndvi_vis, 'NDVI_max', bounds=region.getBounds())
map.addLayer(NDVI_min,ndvi_vis, 'NDVI_min', bounds=region.getBounds())
map.addLayer(wheat,mask_vis, 'wheat', bounds=region.getBounds())    # 绿色区域为小麦

map

Map(center=[33.469984534999995, 116.181687705], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

### 参考文献
*潘力，夏浩铭，王瑞萌，等. 基于Google Earth Engine 的淮河流域越冬作物种植面积制图[J]. 农业工程学报，2021，37(18)：211-218. doi：10.11975/j.issn.1002-6819.2021.18.025 <a href="http://www.tcsae.org" target="_blank">http://www.tcsae.org</a>*


*备注：案例以建立规则集提取冬小麦为主，主要借鉴参考文献中部分分类参数和指标用做流程测试，另仅使用 *Landsat-8* 数据，因此与参考文献中的成果有一定差异敬请谅解。*